In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import scipy.stats
!pip install xgboost
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns

# Data Pre-Processing
In this section, we process the input data sets (categories, ERA5 and EVI), filter for relevant details and prepare the input data.

## Load categories

In [ ]:
categories = pd.read_csv('data/categories.csv')
categories['values'].unique()

## Read urban climate data

In [ ]:
era_5_berlin = pd.read_csv('data/era5_berlin_wide.csv')

In [ ]:
era_5_berlin

## Process ERA5 data
- drop null values, filter for Berlin rasters, aggregate data, drop categorical columns, create date_key as identifier

In [ ]:
# code generation supported by ChatGPT
era_5_berlin = era_5_berlin.dropna()

filtered_era5 = era_5_berlin[era_5_berlin['cell'].isin([6, 7])]

era_5_berlin = filtered_era5.groupby('time').mean(numeric_only=True)
era_5_berlin.drop(columns=["SFC=Ground or water surface; Precipitation type [0=Reserved; 1=Rain; 2=Thunderstorm; 3=Freezing rain; 4=Mixed/ice; 5=Snow; 6=Wet snow; 7=Mixture of rain and snow; 8=Ice pellets; 9=Graupel; 10=Hail; 11=Drizzle; 12=Freezing drizzle; 13-191=Reserved; 192-254=Reserved for local use; 255=Missing]"], inplace=True)
#era_5_berlin.drop(columns=["SFC (Ground or water surface); undefined [-]"], inplace=True)
era_5_berlin.index = pd.to_datetime(era_5_berlin.index)
era_5_berlin['date_key'] = era_5_berlin.index.strftime('%Y_%m')
print(era_5_berlin.head())

## Rename variable names to improve readability

In [ ]:
era_5_berlin.columns
era_5_berlin.rename(columns={'SFC (Ground or water surface); 2 metre temperature [C]': '2 metre temperature'}, inplace=True)
era_5_berlin.rename(columns={'SFC (Ground or water surface); 10m. Windspeed (irresp of dir.) [m/s]': '10m. Windspeed'}, inplace=True)
era_5_berlin.rename(columns={'SFC (Ground or water surface); Total cloud cover (0 - 1) [-]': 'Total cloud cover'}, inplace=True)
era_5_berlin.rename(columns={'SFC (Ground or water surface); Snow depth (water equivalent) [m]': 'Snow depth'}, inplace=True)
era_5_berlin.rename(columns={'SFC (Ground or water surface); Total column ozone Dobson [kg/m^2]': 'Ozone Dobson'}, inplace=True)
era_5_berlin.rename(columns={'SFC (Ground or water surface); Total column water [kg/m^2]': 'Water [kg/m^2]'}, inplace=True)
era_5_berlin.rename(columns={'0-7[cm] DBLY (layer between 2 depths below land surface); Surf.temp/soil temp lev 1 (from 930804) [C]': 'Surf.temp lev 1'}, inplace=True)
era_5_berlin.rename(columns={'7-28[cm] DBLY (layer between 2 depths below land surface); Deep soil tmp/soil temp lev2(from 930804) [C]': 'Deep soil temp lev2'}, inplace=True)
era_5_berlin.rename(columns={'28-100[cm] DBLY (layer between 2 depths below land surface); Clim deep soil tmp/soil tmp lev3(930804) [C]': 'Deep soil tmp lev3'}, inplace=True)
era_5_berlin.rename(columns={'100-255[cm] DBLY (layer between 2 depths below land surface); Soil temperature level 4 [C]': 'Deep soil temperature level 4'}, inplace=True)

era_5_berlin.rename(columns={'0-7[cm] DBLY (layer between 2 depths below land surface); Volumetric soil water layer 1 [m^3/m^3]': 'Volumetric soil water layer 1'}, inplace=True)
era_5_berlin.rename(columns={'7-28[cm] DBLY (layer between 2 depths below land surface); Volumetric soil water layer 2 [m^3/m^3]': 'Volumetric soil water layer 2'}, inplace=True)
era_5_berlin.rename(columns={'28-100[cm] DBLY (layer between 2 depths below land surface); Volumetric soil water layer 3 [m^3/m^3]': 'Volumetric soil water layer 3'}, inplace=True)
era_5_berlin.rename(columns={'100-255[cm] DBLY (layer between 2 depths below land surface); Volumetric soil water layer 4 [m^3/m^3]': 'Volumetric soil water layer 4'}, inplace=True)

era_5_berlin.rename(columns={'SFC (Ground or water surface); Total precipitation [m]': 'Total precipitation'}, inplace=True)
era_5_berlin.rename(columns={'SFC (Ground or water surface); 2 metre temperature [C]': '2 metre temperature'}, inplace=True)
era_5_berlin.rename(columns={'SFC (Ground or water surface); Instantaneous 10 metre wind gust [m/s]': 'Instantaneous 10 metre wind gust'}, inplace=True)
era_5_berlin.rename(columns={'SFC (Ground or water surface); Evaporation (of water) [m]': 'Evaporation (of water)'}, inplace=True)
era_5_berlin.rename(columns={'SFC (Ground or water surface); Runoff [m]': 'Runoff'}, inplace=True)
era_5_berlin.rename(columns={'SFC (Ground or water surface); Snow fall (of water equivalent) [m]': 'Snow fall (of water equivalent)'}, inplace=True)
era_5_berlin.rename(columns={'SFC (Ground or water surface); Snowmelt [m of water equivalent]': 'Snowmelt'}, inplace=True)
era_5_berlin.rename(columns={'SFC (Ground or water surface); undefined [-]': 'Potential Evaporation'}, inplace=True)


era_5_berlin

## Load EVI Data

In [ ]:
evi_aoi = pd.read_csv('data/EVI_AOI.csv')

In [ ]:
evi_aoi

Drop null values, detect and delete outliers

In [ ]:
evi_aoi = evi_aoi.dropna(subset=['values'])

In [ ]:
evi_min = evi_aoi.min()
evi_max = evi_aoi.max()
print("Min:", evi_min)
print("Max:", evi_max)

Remove outliers

In [ ]:
lower_bound = evi_aoi['values'].quantile(0.05)
upper_bound = evi_aoi['values'].quantile(0.95)

evi_aoi = evi_aoi[(evi_aoi['values'] >= lower_bound) & (evi_aoi['values'] <= upper_bound)]

print(f"Lower Bound (5th Percentile): {lower_bound}")
print(f"Upper Bound (95th Percentile): {upper_bound}")

# Merge Datasets and prepare for Training

## Merge categories and EVI dataframe, filter for desired EVI dataset

In [ ]:
def choose_cat_and_merge(df_evi, df_cat, cat):
    df_cat = df_cat[df_cat['values'] == cat]
    df_evi = pd.merge(df_evi, df_cat, on='cell', how='inner')
    return df_evi

## Merge ERA and EVI data

In [ ]:
def merge_with_era5(df_era5, df_evi): 
    df_evi['date_key'] = df_evi['layer_x'].str.extract(r'(\d{4}_\d{2})')
    df_evi['year'] = df_evi['layer_x'].str.extract(r'EVI_(\d{4})').astype(int)
    df_evi = df_evi[df_evi['year'] <= 2021]
    evi_aoi_grouped = df_evi.groupby('date_key', as_index=False)['values_x'].mean()
    df = pd.merge(df_era5, evi_aoi_grouped, on='date_key', how='inner')
    return df


In [ ]:
# Code debugged by ChatGPT
def select_columns(df1, correlation_threshold=0.9):
    corr_matrix = df1.corr().abs()
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > correlation_threshold)]
    df_cleaned = df1.drop(columns=to_drop)    
    print(f"Removed columns due to high collinearity: {to_drop}")
    return df_cleaned



In [ ]:
# Code debugged by ChatGPT
def train_xgb(df):
    df.columns = df.columns.str.replace(r'[^\w\s]', '', regex=True)
    X = df.iloc[:, :-1]  # Select all columns except the last one (features)
    y = df.iloc[:, -1]   # Select only the last column (target)
    split_index = int(0.9 * len(df))
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]
    xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
    tscv = TimeSeriesSplit(n_splits=5)
    param_grid = {
        'n_estimators': [100, 200, 500],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.7, 0.8, 1.0]
    }
    
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                               cv=tscv, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    print("Best Hyperparameters:", best_params)
    best_model = XGBRegressor(objective='reg:squarederror', **best_params, random_state=42)
    best_model.fit(X_train, y_train)
    
    y_pred = best_model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R² Score: {r2:.4f}")

    plt.figure(figsize=(10, 6))
    xgb.plot_importance(best_model, importance_type='weight', title='Feature Importance', height=0.8)
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label="Actual", color='blue')
    plt.plot(y_test.index, y_pred, label="Predicted", color='red', linestyle='dashed')
    plt.xlabel("Time")
    plt.ylabel("Target Value")
    plt.title("Actual vs. Predicted Values")
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 8))
    sns.heatmap(X.corr(), annot=True, cmap="coolwarm", center=0)
    plt.title("Feature Correlation Heatmap")
    plt.show()

    return best_model


In [ ]:
#columns that appear in the areas of interest
df_evi = pd.merge(evi_aoi, categories, on='cell', how='inner')
df_evi['values_y'].unique()

In [ ]:
# Model Training

In [ ]:
# Model allotment gardens
evi_aoi_allotments = choose_cat_and_merge(evi_aoi, categories, 'AX41008_AX_SportFreizeitUndErholungsflaeche__Kleingarten')
results_allotments = merge_with_era5(era_5_berlin,evi_aoi_allotments)
results_allotments = results_allotments.drop(columns=['cell','x','y','date_key','cell_x', 'x_x', 'y_x', 'time', 'cell_y', 'x_y', 'y_y', 'layer_x', 'layer_y'], errors='ignore')
results_allotments = select_columns(results_allotments)
model_allotments = train_xgb(results_allotments)

In [ ]:
# Model semi-natural areas
evi_aoi_vegetationslos = choose_cat_and_merge(evi_aoi, categories, 'AX43007_AX_UnlandVegetationsloseFlaeche__Naturnahe Fläche')
results_vegetationslos = merge_with_era5(era_5_berlin,evi_aoi_vegetationslos)
results_vegetationslos = results_vegetationslos.drop(columns=['cell','x','y','date_key','cell_x', 'x_x', 'y_x', 'time', 'cell_y', 'x_y', 'y_y', 'layer_x', 'layer_y'], errors='ignore')
results_vegetationslos = select_columns(results_vegetationslos)
model_vegetationslos = train_xgb(results_vegetationslos)

In [ ]:
# Model Forest
evi_aoi_forest = choose_cat_and_merge(evi_aoi, categories, 'AX43002_AX_Wald_Laubholz_')
results_forest = merge_with_era5(era_5_berlin,evi_aoi_forest)
results_forest = results_forest.drop(columns=['cell','x','y','date_key','cell_x', 'x_x', 'y_x', 'time', 'cell_y', 'x_y', 'y_y', 'layer_x', 'layer_y'], errors='ignore')
results_forest = select_columns(results_forest)
model_forest = train_xgb(results_forest)

In [ ]:
# Model Park
evi_aoi_sport_freizeit = choose_cat_and_merge(evi_aoi, categories, 'AX41008_AX_SportFreizeitUndErholungsflaeche__Park')
results_sport_freizeit = merge_with_era5(era_5_berlin,evi_aoi_sport_freizeit)

results_sport_freizeit = results_sport_freizeit.drop(columns=['cell','x','y','date_key','cell_x', 'x_x', 'y_x', 'time', 'cell_y', 'x_y', 'y_y', 'layer_x', 'layer_y'], errors='ignore')
results_sport_freizeit = select_columns(results_sport_freizeit)
model_sport_freizeit = train_xgb(results_sport_freizeit)

In [ ]:
# Model Woods
evi_aoi_gehoelz = choose_cat_and_merge(evi_aoi, categories, 'AX54001_AX_Vegetationsmerkmal__Gehölz__')
results_gehoelz = merge_with_era5(era_5_berlin,evi_aoi_gehoelz)
results_gehoelz = results_gehoelz.drop(columns=['cell','x','y','date_key','cell_x', 'x_x', 'y_x', 'time', 'cell_y', 'x_y', 'y_y', 'layer_x', 'layer_y'], errors='ignore')
results_gehoelz = select_columns(results_gehoelz)
model_holz = train_xgb(results_gehoelz)

In [ ]:
# Model Parks
evi_aoi_gruenanlage = choose_cat_and_merge(evi_aoi, categories, 'Grünanlage')
results_gruenanlage = merge_with_era5(era_5_berlin,evi_aoi_gruenanlage)
results_gruenanlage = results_gruenanlage.drop(columns=['cell','x','y','date_key','cell_x', 'x_x', 'y_x', 'time', 'cell_y', 'x_y', 'y_y', 'layer_x', 'layer_y'], errors='ignore')
results_gruenanlage = select_columns(results_gruenanlage)
model_gruenanlage = train_xgb(results_gruenanlage)

In [ ]:
# Model Playgrounds
evi_aoi_spielplatz = choose_cat_and_merge(evi_aoi, categories, 'Spielplatz')
results_spielplatz = merge_with_era5(era_5_berlin,evi_aoi_spielplatz)
results_spielplatz = results_spielplatz.drop(columns=['cell','x','y','date_key','cell_x', 'x_x', 'y_x', 'time', 'cell_y', 'x_y', 'y_y', 'layer_x', 'layer_y'], errors='ignore')
results_spielplatz = select_columns(results_spielplatz)
model__spielplatz = train_xgb(results_spielplatz)

In [ ]:
# Model Game Reserve
evi_aoi_wildgehege = choose_cat_and_merge(evi_aoi, categories, 'AX51006_AX_BauwerkOderAnlageFuerSportFreizeitUndErholung_Wildgehege___')
results_wildgehege = merge_with_era5(era_5_berlin,evi_aoi_wildgehege)
results_wildgehege = results_wildgehege.drop(columns=['cell','x','y','date_key','cell_x', 'x_x', 'y_x', 'time', 'cell_y', 'x_y', 'y_y', 'layer_x', 'layer_y'], errors='ignore')
results_wildgehege = select_columns(results_wildgehege)
model_wildgehege = train_xgb(results_wildgehege)